In [1]:
conda env list


# conda environments:
#
base                   /opt/conda
adsp-genai           * /opt/conda/envs/adsp-genai
interview-ai           /opt/conda/envs/interview-ai
jupyterlab             /opt/conda/envs/jupyterlab
jupyterlab4-preview    /opt/conda/envs/jupyterlab4-preview
pytorch                /opt/conda/envs/pytorch
tensorflow             /opt/conda/envs/tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
# source activate adsp-genai
!source /opt/conda/bin/activate adsp-genai

In [3]:
import os
print("🔹 CONDA env:", os.environ.get("CONDA_DEFAULT_ENV"))

🔹 CONDA env: adsp-genai


In [4]:
# !pip install --quiet --upgrade datasets langchain langchain-community langchain-chroma sentence-transformers umap-learn matplotlib scikit-learn
# !pip install --upgrade PyMuPDF
# !pip install huggingface_hub

In [5]:
# import sys
# print(sys.executable)

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [7]:
df = pd.read_csv("../Data/scraped_output_metadata.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1304 non-null   int64 
 1   text        1304 non-null   object
 2   metadata    1304 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.7+ KB


In [9]:
# df = df.rename(columns= {'scraped_Content': 'scraped_content'})

In [10]:
# df_filtered = df.drop(index = [4, 5, 14, 15, 24, 25])

In [11]:
# df_filtered_reset_index = df_filtered.reset_index()
# df_filtered_reset_index = df_filtered_reset_index.drop(columns = 'index')
# df_filtered_reset_index.info()

In [12]:
# df_filtered_reset_index.sample(2)

In [13]:
# df = df_filtered_reset_index.copy()

In [9]:
pd.set_option("display.max_colwidth", 500)

In [10]:
df.head()

,Unnamed: 0,text,metadata
0,0,"Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section'}"
1,1,"Time Series Analysis and ForecastingTime Series Analysis is a science as well as the art of making rational predictions based on previous records. It is widely used in various fields in today’s business settings.Statistical Models for Data ScienceIn a traditional linear model, the observed response follows a normal distribution, and the expected response value is a linear combination of the predictors. Since Carl Friedrich Gauss (1777-1855) and Adrien-Marie Legendre (1752-1833) created this...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Core Courses', 'super_section': 'Unknown Super Section'}"
2,2,"Advanced Computer Vision with Deep LearningComputer vision is the field of computer science that focuses on creating digital systems that can process, analyze, and make sense of visual data in the same way that humans do. Deep learning is a subset of machine learning and a branch of Artificial Intelligence (AI). It involves the training, deployment, and application of large complex neural network architectures to solve cutting-edge problems. Deep Learning has become the primary approach for ...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Sample Elective Courses', 'super_section': 'Unknown Super Section'}"
3,3,Take the next step to advance your career with UChicago’s MS in Applied Data Science.,"{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Your Career Success', 'super_section': None}"
4,4,"The In-Person program admits full- and part-time students for entrance in Autumn quarter annually. The full-time, in-person MS in Applied Data Science program isSTEM/OPT eligible. Please visit theOnlineprogram page if you are interested in those full- and part-time options.","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Your Career Success', 'super_section': None}"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1304 non-null   int64 
 1   text        1304 non-null   object
 2   metadata    1304 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.7+ KB


In [12]:
import ast

df['metadata'] = df['metadata'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1304 non-null   int64 
 1   text        1304 non-null   object
 2   metadata    1304 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.7+ KB


## Document Indexing

### Chunking

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Iterable, List

def chunk_text(
    text: str,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(text)

def build_chunks_dataframe(
    df: pd.DataFrame,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> pd.DataFrame:
    
    records = []

    for source_idx, row in df.iterrows():
        seg_text = row['text']
        seg_metadata = row['metadata']

        seg_chunks = chunk_text(seg_text, chunk_size, chunk_overlap)
        
        for chunk_idx, chunk in enumerate(seg_chunks):
            meta = seg_metadata.copy()
            meta['source_idx'] = source_idx
            meta['chunk_idx'] = chunk_idx
            
            records.append({'chunk_text': chunk,
                            'metadata': meta})

    return pd.DataFrame(records)

In [15]:
df_chunks = build_chunks_dataframe(df)

In [16]:
df_chunks

,chunk_text,metadata
0,"Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 0}"
1,". 0 units, no cost.R for Data Science (Foundational, optional)This course is held the five weeks leading up to the start of your first quarter and is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.Python for Data Science (Foundational, optional)This course is held concurrently with the first five weeks of your first quarter in the program and starts with an introduction to the Python programming language basic syntax and en...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 1}"
2,". 0 units, no cost.Brush up on the Basics (Optional resource)If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.Four Coursera courses cover very similar topics. You can review the Coursera curricula to see if you are already well-prepared, or if you like, s...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 2}"
3,".Mathematics for Machine Learning: Linear Algebra (offered by University College London)Basis Data Descriptors, Statistical Distributions, and Application to Business Decisions (offered by Rice University)Python for Data Science, AI, & Development (offered by IBM)Data Analysis with R Programming (offered by Google)","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 3}"
4,"Time Series Analysis and ForecastingTime Series Analysis is a science as well as the art of making rational predictions based on previous records. It is widely used in various fields in today’s business settings.Statistical Models for Data ScienceIn a traditional linear model, the observed response follows a normal distribution, and the expected response value is a linear combination of the predictors. Since Carl Friedrich Gauss (1777-1855) and Adrien-Marie Legendre (1752-1833) created this...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Core Courses', 'super_section': 'Unknown Super Section', 'source_idx': 1, 'chunk_idx': 0}"
...,...,...
1519,"An MBA from Booth is a significant investment in yourself, your intellect, your relationships, and your future. Here's how to finance it.","{'source_url': 'https://www.chicagobooth.edu/mba/f

In [17]:
# df['chunk_text'] = df['scraped_content'].apply(lambda x: chunk_text(x, chunk_size = 800, chunk_overlap = 80))

In [18]:
# df_chunks['chunk_count'] = df_chunks['chunk_text'].apply(len)

In [19]:
# from itertools import chain

# all_chunks = list(chain.from_iterable(df_chunks['chunk_text']))

# total_chunks = len(all_chunks)
# total_doc = len(df_chunks)
# print(f"Total Chunk Texts: {total_chunks}")
# print(f"Average Chunk per Document: {(total_chunks / total_doc):.2f}")

In [20]:
all_chunks = df_chunks['chunk_text'].tolist()
all_metas = df_chunks['metadata'].tolist()

In [21]:
len(all_chunks)

1524

In [22]:
all_chunks[:5]

['Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational, optional)This course is held in the five weeks leading up to the start of your first quarter and provides general exposure to basic statistical concepts necessary for success in advanced courses in the program. 0 units, no cost',
 '. 0 units, no cost.R for Data Science (Foundational, optional)This course is held the five weeks leading up to the start of your first quarter and is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.Py

In [23]:
all_metas[:5]

[{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/',
  'retrieved_at': '2025-07-22T20:17:29.207159',
  'title': 'In-Person Program – DSI',
  'section': 'Noncredit Courses',
  'super_section': 'Unknown Super Section',
  'source_idx': 0,
  'chunk_idx': 0},
 {'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/',
  'retrieved_at': '2025-07-22T20:17:29.207159',
  'title': 'In-Person Program – DSI',
  'section': 'Noncredit Courses',
  'super_section': 'Unknown Super Section',
  'source_idx': 0,
  'chunk_idx': 1},
 {'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/',
  'retrieved_at': '2025-07-22T20:17:29.207159',
  'title': 'In-Person Program – DSI',
  'section': 'Noncredit Courses',
  'super_section': 'Unknown Super Section',
  'source_idx': 0,
  'chunk_idx': 2},
 {'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/',
  

### Embedding

In [24]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Setting device to '{device}'")

Setting device to 'cuda'


In [25]:
from typing import List
# Use a pre-trained Sentence-BERT model to convert each chunk into a semantic vector
# embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedder = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

def embed_chunk_docs(chunk_docs: List[str],
                     model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                     normalize: bool = True):
    """
    Embed chunks of documents based on a given model name.
    """
    
    embedder = HuggingFaceEmbeddings(
    model_name= model_name,
    encode_kwargs={"normalize_embeddings": normalize})
    
    embeddings = embedder.embed_documents(chunk_docs)
    
    return embedder, embeddings

In [45]:
# embedder, embeddings = embed_chunk_docs(all_chunks, "intfloat/e5-large-v2", True)
# embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/all-MiniLM-L6-v2", True)
embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/paraphrase-MiniLM-L6-v2", True)

In [46]:
len(embeddings)

1524

In [47]:
len(embeddings[0])

384

### Store Embeddings in Chroma DB

In [34]:
# !rm -rf ./chroma_scifact

In [48]:
from langchain_chroma import Chroma
# from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
import time
from pathlib import Path
from typing import Optional, Union


def store_embed_in_db(texts: List[str],
    metas: List[dict],
    embedder: Optional[Embeddings] = None,
    persist_directory: Optional[Union[str, Path]] = None
) -> Chroma:
    """
    Store text chunks (and their embeddings) in a Chroma vector database.

    You must provide *either* `embeddings` (precomputed) *or* an `embedder` 
    (model that implements .embed_documents).

    Args:
        texts: List of text chunks to index.
        embedder: Optional Embeddings object; used to compute embeddings on the fly.
        persist_directory: Directory where the Chroma DB will be saved. 
            If None, a timestamped folder will be created.

    Returns:
        A Chroma vector store instance with your texts indexed.
    """
        
    if embedder is None:
        raise ValueError("You must supply an `embedder`")
        
    if persist_directory is None:
        ts = int(time.time())
        persist_directory = Path(f"./chroma_scifact_run_{ts}")
    persist_directory = Path(persist_directory)
    
    all_metas = df_chunks['metadata'].tolist()
    
    
    db = Chroma.from_texts(
        texts=texts,
        embedding=embedder,
        metadatas = metas,
        persist_directory=str(persist_directory)
    )
        
    print("Chunks successfully stored in Chroma vector DB.")
    
    return db

In [49]:
db = store_embed_in_db(all_chunks, all_metas, embedder = embedder)

Chunks successfully stored in Chroma vector DB.


In [50]:
col = db._collection  # the underlying chromadb Collection
data = col.get() 

In [51]:
len(data['documents'])

1524

In [52]:
meta_section_set = set()

for meta in all_metas:
    meta_section_set.add(meta.get('section', ''))

In [53]:
meta_section_set

{'#UCHICAGO2025',
 '2024-2025 LIVING EXPENSES & FEES(based on half-time/full-time enrollment)',
 '2025-2026 LIVING EXPENSES & FEES(based on half-time/full-time enrollment)',
 'A Chicago Booth MBA offers unrivaled return on investment for talented, ambitious students.',
 'A day at Google for UChicago’s MS in Applied Data Science students',
 'Additional Resources',
 'Admissions Process',
 'Advanced Computer Vision with Deep Learning',
 'Advanced Linear Algebra for Machine Learning (Foundational)Optional',
 'Advanced Linear Algebra for Machine Learning (Foundational, optional)',
 'Advanced Machine Learning and Artificial Intelligence',
 'Alumni & Friends | Physical Sciences Division | The University of Chicago',
 'An NFL Ticket Pricing Study: Optimizing Revenue Using Variable And Dynamic Pricing Methods',
 'Application Fee',
 'Application Fee Waivers',
 'Application Management',
 'Application Process',
 'Application Requirements | Physical Sciences Division | The University of Chicago',
 

In [54]:
meta_super_section_set = set()

for meta in all_metas:
    meta_super_section_set.add(meta.get('super_section', ''))

In [55]:
meta_super_section_set

{'Alumni & Friends',
 'Application Requirements',
 'By and For Data Science Innovators',
 'Cost of Attendance',
 'Curriculum',
 'Full-Time MBA Cost',
 'Interested in Becoming a Capstone Sponsor?',
 'MBA/MS in Applied Data Science',
 'MS in Applied Data Sciencefacet-arrow-down',
 'Master’s in Applied Data Science Application Requirements',
 'Meet your Admissions Counselor',
 'Noncredit Courses',
 None,
 'Program Structure',
 'Recorded Admissions EventsJoint Degree Programs',
 'Selected Capstone Projects',
 'The Data Science Institute Scholarship, MS in Applied Data Science Alumni Scholarship',
 'The University of Chicago: Graduate Studies',
 'Unknown Super Section',
 'Upcoming Events'}

In [56]:
df_chunks[df_chunks['metadata'].apply(lambda x: x.get('section', '') == 'Noncredit Courses')]

,chunk_text,metadata
0,"Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 0}"
1,". 0 units, no cost.R for Data Science (Foundational, optional)This course is held the five weeks leading up to the start of your first quarter and is an introduction to the essential concepts and techniques for the statistical computing language R. 0 units, no cost.Python for Data Science (Foundational, optional)This course is held concurrently with the first five weeks of your first quarter in the program and starts with an introduction to the Python programming language basic syntax and en...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 1}"
2,". 0 units, no cost.Brush up on the Basics (Optional resource)If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.If you would like to gauge your preparation in Foundational course topics, we recommend specific Coursera courses that cover very similar topics.Four Coursera courses cover very similar topics. You can review the Coursera curricula to see if you are already well-prepared, or if you like, s...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 2}"
3,".Mathematics for Machine Learning: Linear Algebra (offered by University College London)Basis Data Descriptors, Statistical Distributions, and Application to Business Decisions (offered by Rice University)Python for Data Science, AI, & Development (offered by IBM)Data Analysis with R Programming (offered by Google)","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/in-person-program/', 'retrieved_at': '2025-07-22T20:17:29.207159', 'title': 'In-Person Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 0, 'chunk_idx': 3}"
948,"Career Seminar (Seminar, required)The Pass/Fail Career Seminar supports the development of industry professional skills, job and/or internship searches, and other in-demand areas of competency among today’s employers. Students enroll in the Career Seminar each quarter in order to engage in unique content throughout their degree program. Students with significant full-time work experience may be eligible to waive this course. 0 units, no cost.Introduction to Statistical Concepts (Foundational...","{'source_url': 'https://datascience.uchicago.edu/education/masters-programs/online-program/', 'retrieved_at': '2025-07-22T20:17:34.238124', 'title': 'Online Program – DSI', 'section': 'Noncredit Courses', 'super_section': 'Unknown Super Section', 'source_idx': 815, 'chunk_idx': 0}"
949,". 0 units, no cost.R for Data Science (Foundational, optional)This Foundational, optional course is held in the 5 weeks leading up to the start of your first quarter. This course is an introduction t

In [57]:
# all_metas.get('section', '')

## Query-Response Pipeline (RAG)

In [58]:
# Improved retrieval function with query expansion and better ranking
from langchain.vectorstores.base import VectorStoreRetriever
import re
from typing import List, Tuple, Dict, Any

def expand_query(query: str) -> List[str]:
    """Expand query with related terms for better retrieval"""
    expansions = {
        "core courses": ["required courses", "core curriculum", "fundamental courses", 
                        "mandatory courses", "core classes", "required classes"],
        "admission requirements": ["application requirements", "prerequisites", 
                                 "eligibility", "admission criteria"],
        "capstone": ["final project", "thesis", "culminating project", "capstone project"],
        "curriculum": ["courses", "program structure", "degree requirements"],
        "machine learning": ["ML", "artificial intelligence", "AI", "deep learning"],
        "data science": ["analytics", "data analytics", "data mining", "statistics"]
    }
    
    expanded_queries = [query]
    query_lower = query.lower()
    
    for key, synonyms in expansions.items():
        if key in query_lower:
            for synonym in synonyms:
                expanded_queries.append(query.replace(key, synonym, 1))
    
    return expanded_queries

def detect_metadata_filter(query: str) -> Optional[Dict[str, Any]]:
    """
    Look for trigger phrases in the query and return an appropriate
    metadata filter for your vector store.
    """
    q = query.lower()
    mapping = {
        "core course":     {"section": {"$eq": "Core Courses"}},
        "elective":        {"section": {"$eq": "Sample Elective Courses"}},
        "foundation":      {"section": {"$eq": "Noncredit Courses"}},
        "capstone sponsor":{"section": {"$eq": "MS-ADS Capstone Sponsor Guide 2025"}},
        "application":     {"super_section": {"$eq": "Master’s in Applied Data Science Application Requirements"}},
        "admission":       {"super_section": {"$eq": "Master’s in Applied Data Science Application Requirements"}}

    }

    for trigger, filt in mapping.items():
        if trigger in q:
            return filt
    return None


def retrive_top_k_docs_improved(selected_query: str, 
                               db_retriever: VectorStoreRetriever,
                               embedder: Embeddings,
                               k_docs: int = 10,  # Increased from 5
                               lambda_mult: float = 0.3,  # Lower for more similarity focus
                               query_prefix: str = "query: ",
                               doc_prefix: str = "passage: ",
                               use_query_expansion: bool = True):
    """
    Improved retrieval with query expansion and better re-ranking
    """
    
    if embedder is None:
        raise ValueError("You must supply an `embedder`.")
    
    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]
    
    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)
    
    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []
    
    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"
        
        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr", 
            search_kwargs={"k": k_docs, 
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )
        
        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)
        
        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))
    
    # Step 3: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))
    
    # Step 4: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)
    
    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]
    
    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)
    
    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]
    
    # Step 5: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))
    
    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)
    
    # Step 6: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))
    
    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))
        
        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))
    
    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)
    
    # Display results
    print("Query:", selected_query)
    print(f"\nExpanded to {len(expanded_queries)} queries")
    print(f"Retrieved {len(unique_docs)} unique documents")
    print(f"\nTop {k_docs} Retrieved Chunks with Scores:\n")
    
    final_results = []
    for i, (doc, score, exp_query) in enumerate(boosted_docs[:k_docs]):
        print(f"Chunk {i+1} (Score: {score:.4f}, From: '{exp_query}'):")
        print(f"{doc}")
        print(f"{'-'*80}")
        final_results.append((doc, score))
        
    return boosted_docs[:k_docs]

In [64]:
# Compare original vs improved retrieval with question selection
QUERIES = {"1": "What are the core courses in the MS in Applied Data Science program?",
           "2": "What are the admission requirements for the MS in Applied Data Science program?",
           "3": "Can you provide information about the capstone project?"}

print("Available queries:")
for key, value in QUERIES.items():
    print(f"{key}: {value}")

query_choice = input("Enter number of your query choice: ")
selected_query = QUERIES.get(query_choice)

if selected_query:
    print(f"\nSelected query: {selected_query}")
    
    print("\n" + "=" * 50)
    print("IMPROVED RETRIEVAL METHOD")
    print("=" * 50)
    
    result_improved = retrive_top_k_docs_improved(selected_query, 
                       db_retriever = db,
                       embedder = embedder, 
                       k_docs = 10, 
                       lambda_mult = 0.3,
                       query_prefix = 'query: ',
                       doc_prefix = 'passage: ',
                       use_query_expansion = True)
else:
    print("Invalid choice. Please select 1, 2, or 3.")

Available queries:
1: What are the core courses in the MS in Applied Data Science program?
2: What are the admission requirements for the MS in Applied Data Science program?
3: Can you provide information about the capstone project?


Enter number of your query choice:  1



Selected query: What are the core courses in the MS in Applied Data Science program?

IMPROVED RETRIEVAL METHOD
Query: What are the core courses in the MS in Applied Data Science program?

Expanded to 11 queries
Retrieved 6 unique documents

Top 10 Retrieved Chunks with Scores:

Chunk 1 (Score: 0.6268, From: 'What are the core courses in the MS in Applied Data Science program?'):
passage: .Leadership and Consulting for Data ScienceThe Leadership and Consulting for Data Scientist course is focused on:• Learning techniques and proven methods to effectively grasp the business domain including organizational dynamics of consultancies and client organizations• Developing relevant solutions to enterprise problems using the sampling methods, traditional statistical techniques and modern machine learning models that deliver value to the organization• Practicing successful project delivery through effective data discovery, influential team membership and leadership, project management, and com

In [61]:
from langchain.vectorstores.base import VectorStoreRetriever

def create_rag_prompt(selected_query: str, 
                       db_retriever: VectorStoreRetriever,
                       embedder: Embeddings,
                       k_docs: int = 10,
                       lambda_mult: float = 0.3,
                       query_prefix: str = "query: ",
                       doc_prefix: str = "passage: ",
                       use_query_expansion: bool = True):
    """
    Retrieve the top-k most relevant chunks for a given query using
    Maximal Marginal Relevance, then re-rank by cosine similarity.

    Args:
        query: Raw query string.
        db_retriever: The `db.as_retriever(...)` object configured with MMR.
        embedder: An Embeddings instance (implements embed_query & embed_documents).
        k: How many chunks to return.
        lambda_mult: MMR diversity parameter (0→pure similarity, 1→pure diversity).
        query_prefix: Text to prepend before embedding the query.
        doc_prefix: Text to prepend before embedding each chunk.

    Returns:
        A list of (chunk_text, score) sorted descending by score.
    """
    
    if embedder is None:
        raise ValueError("You must supply an `embedder`.")
    
    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]
    
    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)
    
    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []
    
    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"
        
        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr", 
            search_kwargs={"k": k_docs, 
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )
        
        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)
        
        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))
    
    # Step 4: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))
    
    # Step 5: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)
    
    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]
    
    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)
    
    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]
    
    # Step 6: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))
    
    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)
    
    # Step 7: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))
    
    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))
        
        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))
    
    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)

    # Combine the retrieved context into a single block of text
    context = "\n".join([doc for doc, score, exp_query in boosted_docs])

    # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""
    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    {context}

    [QUESTION]
    {formatted_query}

    Summarize your answers that are easy to follow. Feel free to use a combination of paragraph and bullet points for your answers
    """

    # Display the final composed prompt that will be passed to the LLM
    print("=== RAG Augmented Prompt ===")
    print(rag_prompt)
        
    return rag_prompt


In [62]:
# Define a new query
QUERIES = {"1": "What are the core courses in the MS in Applied Data Science program?",
 "2" : "What are the admission requirements for the MS in Applied Data Science program?",
 "3" : "Can you provide information about the capstone project?"
}

query_choice = input("Enter number of your query choice")
selected_query = QUERIES.get(query_choice)

rag_prompt = create_rag_prompt(selected_query, 
                   db_retriever = db,
                   embedder = embedder, 
                   k_docs = 10, 
                   lambda_mult = 0.3,
                   query_prefix = 'query: ',
                   doc_prefix = 'passage: ',
                   use_query_expansion = True)

Enter number of your query choice 3


=== RAG Augmented Prompt ===

    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    passage: .Data Science Capstone ProjectThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
passage: . A list of sample electives (subject to change) appear at the bottom of theIn-PersonandOnline Programpages.CapstoneData Science Capstone ProjectLetter GradeThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time stu

In [40]:
# !pip install huggingface_hub
# huggingface-cli login

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

MODELS = {"1": {"model_name":"google/flan-t5-base", "model_type": "encoder-decoder"},
          "2": {"model_name":"mistralai/Mistral-7B-Instruct-v0.3", "model_type": "decoder only"},
          "3": {"model_name":"meta-llama/Llama-2-7b-hf", "model_type": "decoder only"}}

MODEL_CONFIG = {"encoder-decoder": {"model_cls": "text2text-generation",
                                    "pipeline": AutoModelForSeq2SeqLM},
                "decoder only": {"model_cls": "text-generation", 
                                 "pipeline": AutoModelForCausalLM}}

In [42]:
def load_llm_model(selected_model: dict,
                   max_tokens: int = 1024):
    model_name = selected_model['model_name']
    model_type = selected_model['model_type']
    
    model_config_cls = MODEL_CONFIG[model_type]['model_cls']
    model_config_pipeline = MODEL_CONFIG[model_type]['pipeline']
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = model_config_pipeline.from_pretrained(model_name,
                                            device_map="auto",           
                                            torch_dtype="auto")
    
    pipe = pipeline(model_config_cls,
                    model = model,
                    tokenizer = tokenizer,
                    max_new_tokens = max_tokens)
    llm = HuggingFacePipeline(pipeline = pipe)
    
    return llm

In [43]:
def main():
    choice = input("Enter number of your LLM model choice")
    selected_model = MODELS.get(choice)
    
    llm = load_llm_model(selected_model)
    
    query_choice = input("Enter number of your query choice: ")
    selected_query = QUERIES.get(query_choice)
    
    rag_prompt = create_rag_prompt(selected_query, 
                   db_retriever = db,
                   embedder = embedder, 
                   k_docs = 10, 
                   lambda_mult = 0.3,
                   query_prefix = 'query: ',
                   doc_prefix = 'passage: ',
                   use_query_expansion = True
                   )
    
    response = llm(rag_prompt)
    print("\n=== Generated Answer ===")
    print(response)

## Question 1: What are the core courses in the MS in Applied Data Science program?

In [57]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [58]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Enter number of your query choice:  1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAG Augmented Prompt ===

    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    passage: . These methods allowed working with variables that span a broader variety of domains and probability distributions. Besides, methods for the analysis of general associations were developed that are different from the Pearson correlation.Machine Learning IThis course is aimed at providing students an introduction to machine learning with data mining techniques and algorithms. It gives a rigorous methodological foundation in analytical and software tools to successfully undertake projects in Data Science. Students are exposed to concepts of exploratory analyses for uncovering and detecting patterns in multivariate data, hypothesizing and detecting relationship

## Question 2: What are the admission requirements for the MS in Applied Data Science program?

In [55]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


Enter number of your query choice:  2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAG Augmented Prompt ===

    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    passage: The MS in Applied Data Science requires short, recorded videos to a given prompt(s) within the application. These short videos give applicants an alternative means to outline why they are a strong fit for the MS in Applied Data Science program.
passage: The MS in Applied Data Science program requires two letters of recommendation. Westrongly recommendthat at least one letter is written by someone such as a direct manager/supervisor or internship supervisor who can attest to skills you demonstrated or gained though a professional workplace experience (e.g., leadership, teamwork, collaboration, initiative, management, other).
passage: The University of ChicagoA

## Question 3: Can you provide information about the capstone project??

In [63]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


Enter number of your query choice:  3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAG Augmented Prompt ===

    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    passage: .Data Science Capstone ProjectThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
passage: . A list of sample electives (subject to change) appear at the bottom of theIn-PersonandOnline Programpages.CapstoneData Science Capstone ProjectLetter GradeThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time stu